In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
# change to wav file
import subprocess

def convert_mp4_to_wav(input_file, output_file):
    command = ['ffmpeg', '-i', input_file, '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', output_file]
    subprocess.call(command)

input_file = '/content/drive/MyDrive/video.mp4'
output_file = '/content/drive/MyDrive/output.mp4'

convert_mp4_to_wav(input_file, output_file)




In [ ]:
#segment audio every 10 seconds
from pydub import AudioSegment
import os
from speechbrain.pretrained import EncoderASR

audio_file = AudioSegment.from_file("/content/drive/MyDrive/output.mp4", format="wav")

segment_length = 10000
segments = []
for i in range(0, len(audio_file), segment_length):
   segment = audio_file[i:i+segment_length]
   segments.append(segment)


for i, segment in enumerate(segments):
  segment.export(f"Segments/segment_{i}.wav", format="wav")

In [ ]:
# English Audio to English text, then to Amharic translation
from speechbrain.pretrained import EncoderDecoderASR
import os
from googletrans import Translator

asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-wav2vec2-commonvoice-en", savedir="pretrained_models/asr-wav2vec2-commonvoice-en")

dir_path = "Segments"
text_dir = "Texts"
english_text_dir = "English_text"

os.makedirs(text_dir, exist_ok=True)
os.makedirs(english_text_dir, exist_ok=True)

def translate_to_amharic(text):
    translator = Translator(service_urls=['translate.google.com'])
    translation = translator.translate(text, src='en', dest='am')
    translated_text = translation.text
    return translated_text

for i, filename in enumerate(sorted(os.listdir(dir_path))):
    if filename.endswith(".wav"):
        file_path = os.path.join(dir_path, filename)
        transcript = asr_model.transcribe_file(file_path)
        amharic_text = translate_to_amharic(transcript)
        segment_name = os.path.splitext(filename)[0]

        # Saves Amharic text
        amharic_text_file_name = f"text_{segment_name}.txt"
        amharic_text_file_path = os.path.join(text_dir, amharic_text_file_name)

        with open(amharic_text_file_path, "w") as file:
            file.write(amharic_text)
        print(f"Amharic transcription for {filename} saved in {amharic_text_file_name}")

        # Saves English text
        english_text_file_name = f"english_text_{segment_name}.txt"
        english_text_file_path = os.path.join(english_text_dir, english_text_file_name)

        with open(english_text_file_path, "w") as file:
            file.write(transcript)
        print(f"English transcription for {filename} saved in {english_text_file_name}")

Amharic transcription for segment_0.wav saved in text_segment_0.txt
English transcription for segment_0.wav saved in english_text_segment_0.txt
Amharic transcription for segment_1.wav saved in text_segment_1.txt
English transcription for segment_1.wav saved in english_text_segment_1.txt
Amharic transcription for segment_2.wav saved in text_segment_2.txt
English transcription for segment_2.wav saved in english_text_segment_2.txt
Amharic transcription for segment_3.wav saved in text_segment_3.txt
English transcription for segment_3.wav saved in english_text_segment_3.txt
Amharic transcription for segment_4.wav saved in text_segment_4.txt
English transcription for segment_4.wav saved in english_text_segment_4.txt
Amharic transcription for segment_5.wav saved in text_segment_5.txt
English transcription for segment_5.wav saved in english_text_segment_5.txt


In [ ]:
#Amharic text to speech
from ttsmms import TTS
from IPython.display import Audio
from google.colab import files
from scipy.io import wavfile
import numpy as np
import os

tts = TTS("data/amh")


folder_path = "/content/Texts"


for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):

        with open(os.path.join(folder_path, file_name), "r") as file:
            text = file.read()

        wav = tts.synthesis(text)


        audio_data = np.int16(wav["x"] * 32767)

        audio_file_name = "Audios/audio_" + file_name.replace("text_segment", "").replace(".txt", "") + ".wav"

        wavfile.write(audio_file_name, wav["sampling_rate"], audio_data)

        files.download(audio_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Combine the Amharic audio files to a single audio file
import os
import wave
import audioop


audio_folder = "/content/Audios/"


audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]

audio_files.sort()


combined_audio = None
frame_rate = None
sample_width = None

for audio_file in audio_files:

    with wave.open(os.path.join(audio_folder, audio_file), "rb") as audio:

        if combined_audio is None:
            frame_rate = audio.getframerate()
            sample_width = audio.getsampwidth()
            combined_audio = wave.open("combined_audio.wav", "wb")
            combined_audio.setnchannels(audio.getnchannels())
            combined_audio.setsampwidth(sample_width)
            combined_audio.setframerate(frame_rate)

        combined_audio.writeframes(audio.readframes(audio.getnframes()))

combined_audio.close()

print("Combined audio saved as 'combined_audio.wav'.")

Combined audio saved as 'combined_audio.wav'.


In [ ]:
#Speed up audio if needed
import os
from pydub import AudioSegment

input_file = '/content/combined_audio.wav'
output_file = 'output_audio.wav'

audio = AudioSegment.from_file(input_file)

speed_factor = 1.35
speed_up_audio = audio.speedup(playback_speed=speed_factor)

speed_up_audio.export(output_file, format='wav')

<_io.BufferedRandom name='output_audio.wav'>

In [ ]:
#mute the original video
!ffmpeg -i "/content/drive/MyDrive/video.mp4" -c copy -an "muted_video.mp4"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
#Combine the mute video with the Amharic audio
import os
from moviepy.editor import VideoFileClip, AudioFileClip


video_path = "/content/muted_video.mp4"

audio_path = "/content/output_audio.wav"

output_directory = "/content/output/"
output_filename = "video_with_audio.mp4"
output_path = os.path.join(output_directory, output_filename)

os.makedirs(output_directory, exist_ok=True)


video = VideoFileClip(video_path)

audio = AudioFileClip(audio_path)


video = video.set_audio(audio)

video.write_videofile(output_path, codec="libx264", audio_codec="aac")

Moviepy - Building video /content/output/video_with_audio.mp4.
MoviePy - Writing audio in video_with_audioTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/output/video_with_audio.mp4



Moviepy - Done !
Moviepy - video ready /content/output/video_with_audio.mp4


In [ ]:
!pip install ttsmms
!curl https://dl.fbaipublicfiles.com/mms/tts/amh.tar.gz --output amh.tar.gz
!mkdir -p data && tar -xzf amh.tar.gz -C data/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 45.6 MB/s eta 0:00:00
  Created wheel for monotonic-align: filename=monotonic_align-1.0.0-cp310-cp310-linux_x86_64.whl size=305266 sha256=e7c4208a7a271e02e3e5aa9ca39ce0b4f02f1313bec8d6b950b047ec21c13c4b
  Stored in directory: /root/.cache/pi

In [ ]:
!pip install pygoogletranslation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 27.3 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=ea699208fd82b79825a1f8b6f9c9615bb068623c4e945643b006b836baaef995
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [ ]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=3b71a53bda4024c792c7122ac376128547db860809c1d915e24c31f7ace88d7f
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling ch

In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00


In [ ]:
!pip install pydub
!pip install speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.8/553.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 47.2 MB/s eta 0:00:00


In [ ]:
!pip install subprocess

ERROR: Could not find a version that satisfies the requirement subprocess (from versions: none)
ERROR: No matching distribution found for subprocess
